In [1]:
#read in original csvs
import pandas as pd
import numpy as np
from dateutil.parser import parse

#import and read data
samples = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/AmostrasAngolaTerrario.xlsx")
analyses = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Horizontes Analises.xlsx")
morphology = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Horizontes_Morfologia.xlsx")
profile_loc = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Perfis_local.xlsx")
soil_profile = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Perfis_solo.xlsx")
elemental_analyses = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Data XRF Angola_inicial.xlsx")
#soil_type = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_solo.xlsx")

## Cleaning samples data (AmostrasAngolaTerrario)

In [2]:
# Rename columns for clarity
samples.rename(columns={
    'Registo': 'sample_id',
    'Nº Campo': 'site_info_id',
    'Ano': 'year',
    'Perfil': 'profile',
    'Campanha': 'campaign',
    'Colónia_Pais': 'country',
    'Distrito': 'district',
    'AmostraCrivada': 'sample_sifted',
    'AmostraNaoCrivada': 'sample_not_sifted',
    'Prateleira': 'shelf',
    'Sala': 'room'
}, inplace=True)

# Drop unused columns
samples_cleaning = samples.drop(columns=[
    'campaign', 'country', 'Província', 'sample_not_sifted', 'sample_sifted', 'Obs'
], errors='ignore')

# # Add a new Primary Key ID column starting from 1
# samples_cleaning.insert(0, 'sample_id', range(1, len(samples_cleaning) + 1))

# This will show all rows that are duplicates (keep=False shows all duplicates, not just subsequent ones)
duplicates1 = samples[samples.duplicated('sample_id',keep=False)]
print(duplicates1)

# Add empty FK columns
samples_cleaning['lab_info_id'] = pd.NA
samples_cleaning['morpho_id'] = pd.NA
samples_cleaning['site_info_id'] = pd.NA

Empty DataFrame
Columns: [sample_id, site_info_id, year, profile, campaign, country, district, sample_sifted, Província, sample_not_sifted, shelf, room, Obs]
Index: []


In [3]:
samples_cleaning.head()

,sample_id,site_info_id,year,profile,district,shelf,room,lab_info_id,morpho_id
0,630,<NA>,1946.0,139,Huambo,1,22,<NA>,<NA>
1,631,<NA>,1946.0,139,Huambo,1,22,<NA>,<NA>
2,632,<NA>,1946.0,139,Huambo,1,22,<NA>,<NA>
3,633,<NA>,1946.0,139,Huambo,1,22,<NA>,<NA>
4,687,<NA>,1946.0,208,Huambo,1,22,<NA>,<NA>


In [4]:
# Ensure consistent types and formatting
samples_cleaning['profile'] = samples_cleaning['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]
samples_cleaning['shelf'] = samples_cleaning['shelf'].astype("string")

#formatting site_info_id
samples_cleaning['site_info_id'] = samples_cleaning['site_info_id'].astype(str).str.replace('/', '_').str.strip().str[:20]
#formatting morpho_id
samples_cleaning['morpho_id'] = samples_cleaning['morpho_id'].astype(str).str.strip().str[:20]
#formatting lab_info_id
samples_cleaning['lab_info_id'] = samples_cleaning['lab_info_id'].astype(str).str.strip().str[:20]
# formatting year
samples_cleaning['year'] = samples_cleaning['year'].apply(
    lambda x: str(int(x)) if pd.notnull(x) else ''
)

# Convert sample_id to string (no truncation unless necessary)
samples_cleaning['sample_id'] = samples_cleaning['sample_id'].astype(str).str.strip()

# Check for duplicates AFTER conversion
duplicate_ids = samples_cleaning[samples_cleaning.duplicated('sample_id', keep=False)]
print("🔍 Duplicate sample_id values:")
print(duplicate_ids[['sample_id']])

samples_cleaning1 = samples_cleaning



🔍 Duplicate sample_id values:
Empty DataFrame
Columns: [sample_id]
Index: []


In [5]:
samples_cleaning1.dtypes

sample_id               object
site_info_id            object
year                    object
profile                 object
district                object
shelf           string[python]
room                    object
lab_info_id             object
morpho_id               object
dtype: object

In [6]:
# Reorder to match DB schema
samples_check = samples_cleaning1[[
    'sample_id',
    'site_info_id',
    'morpho_id',
    'lab_info_id',
    'profile',
    'year',
    'shelf',
    'room'
]]

samples_check.head()

,sample_id,site_info_id,morpho_id,lab_info_id,profile,year,shelf,room
0,630,<NA>,<NA>,<NA>,139,1946,1,22
1,631,<NA>,<NA>,<NA>,139,1946,1,22
2,632,<NA>,<NA>,<NA>,139,1946,1,22
3,633,<NA>,<NA>,<NA>,139,1946,1,22
4,687,<NA>,<NA>,<NA>,208,1946,1,22


In [8]:
samples_check.to_csv("/Users/inesschwartz/Desktop/samples_check.csv", index=False)

# Cleaning Soil Profile Table (dropping this table)

In [24]:
##rename columns for clarity add FK/PK
#create copy of samples
soil_profile_record = samples.copy()
# Rename columns in the DataFrame
samples.rename(columns={
    'Registo': 'sample_id',
    'Nº Campo': 'site_info_id',
    'Ano': 'year',
    'Perfil': 'profile',
    'Campanha': 'campaign',
    'Colónia_Pais': 'country',
    'Distrito': 'district',
    'AmostraCrivada': 'sample_sifted',
    'AmostraNaoCrivada': 'sample_not_sifted',
    'Prateleira': 'shelf',
    'Sala': 'room'
}, inplace=True)

soil_profile_cleaning2 = soil_profile_record
# Add missing columns
soil_profile_cleaning2['site_info_id'] = pd.NA
soil_profile_cleaning2['sample_id'] = pd.NA 
soil_profile_cleaning2['soil_type_id'] = pd.NA 
soil_profile_cleaning2['site_info_id'] = soil_profile_cleaning2['site_info_id'].astype(str)

# Add a new Primary Key ID column starting from 1
# Check if 'lab_sample_id' exists and drop it
if 'profile_record_id' in soil_profile_cleaning2.columns:
    soil_profile_cleaning2 = soil_profile_cleaning2.drop(columns=['profile_record_id'])

# Insert a new column, e.g., 'lab_sample_id' as a primary key starting from 1
soil_profile_cleaning2.insert(0, 'profile_record_id', range(1, len(soil_profile_cleaning2) + 1))

# Reorder columns to match SAMPLES schema
profile_record_clean = soil_profile_cleaning2[[
    'profile_record_id',
    'profile',
    'site_info_id',
    'sample_id',
    'soil_type_id'
]].copy()

In [25]:
# ensure consistent data types and formatting

# correct datatypes
#profile to string
profile_record_clean['profile'] = profile_record_clean['profile'].astype(str).str.strip().str[:20]

#sample_id to string
# Convert cleaned values to numeric safely
# Convert sample_id to numeric first (coerce errors to NaN)
profile_record_clean['sample_id'] = pd.to_numeric(profile_record_clean['sample_id'], errors='coerce')
# Then convert to string (will keep NaNs as <NA>)
profile_record_clean['sample_id'] = profile_record_clean['sample_id'].astype('Int64').astype(str)


In [26]:
# check for duplicates and nulls
# Filter out and print duplicated samples (including the first occurrence)
duplicated_values = profile_record_clean['sample_id'][profile_record_clean['sample_id'].duplicated()].unique()
print(duplicated_values)

['<NA>']


In [27]:
# Preview the result
profile_record_clean.head()
#profile_record_clean.to_csv("/Users/inesschwartz/Desktop/profile_record_clean.csv", index=False)

,profile_record_id,profile,site_info_id,sample_id,soil_type_id
0,1,139,<NA>,<NA>,<NA>
1,2,139,<NA>,<NA>,<NA>
2,3,139,<NA>,<NA>,<NA>
3,4,139,<NA>,<NA>,<NA>
4,5,208,<NA>,<NA>,<NA>


# Cleaning Site info table (this gives info on location of soil profile/pit and its characteristics)

In [7]:
import unicodedata

site_info_cleaning = profile_loc

site_info_cleaning.rename(columns={
    'PERFIL': 'profile',
    'X_COORD': 'X_coord',
    'Y_COORD': 'Y_coord', 
    'PRO': 'district'
}, inplace=True)

# Add a new Primary Key ID column starting from 1
site_info_cleaning.insert(0, 'site_info_id', range(1, len(site_info_cleaning) + 1))


# Add missing columns
site_info_cleaning['land_cover_id'] = pd.NA
site_info_cleaning['climate_id'] = pd.NA  
site_info_cleaning['geology_id'] = pd.NA
site_info_cleaning['topo_feature_id'] = pd.NA  
#site_info_cleaning['sampling_date'] = pd.NA # might leave out...
site_info_cleaning['soil_type_id'] = pd.NA  
#site_info_cleaning['sample_id'] = pd.NA  # don't want this, creates a many to one relationship


# Function to remove accents
def remove_accents(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in text if not unicodedata.combining(c))
    return text

# Apply to all object (string) columns
for col in site_info_cleaning.select_dtypes(include='object').columns:
    site_info_cleaning[col] = site_info_cleaning[col].apply(remove_accents)

# Replace / with _ and strip/shorten 'profile' BEFORE slicing into site_info_clean
site_info_cleaning['profile'] = (
    site_info_cleaning['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str[:20]
)

# Then select columns
site_info_clean = site_info_cleaning[[
    'site_info_id',
    'ID',
    'profile',
    # 'sampling_date', # maybe?
    'X_coord',
    'Y_coord',
    'district',
    #'land_cover_id',
    'climate_id',
    'geology_id',
    'topo_feature_id',
    'soil_type_id'
    #'sample_id'

]]


site_info_clean.head()

,site_info_id,ID,profile,X_coord,Y_coord,district,climate_id,geology_id,topo_feature_id,soil_type_id
0,1,2770,1_57,12.161278,-15.222598,Namibe,<NA>,<NA>,<NA>,<NA>
1,2,48,1_59,12.575775,-4.866986,Cabinda,<NA>,<NA>,<NA>,<NA>
2,3,1618,1_61,15.098840,-11.225411,Cuanza Sul,<NA>,<NA>,<NA>,<NA>
3,4,881,1_63,17.081955,-9.274587,Malanje,<NA>,<NA>,<NA>,<NA>
4,5,1750,1_64,20.788116,-11.568683,Moxico,<NA>,<NA>,<NA>,<NA>


In [8]:
site_info_clean.to_csv("/Users/inesschwartz/Desktop/site_info.csv", index=False)

# Morphology Horizon Table

In [8]:
# rename columns
morphology.rename(columns={
    'Morfo_id':'morpho_id',
    'Amostra': 'sample_id',
    'Perfil': 'profile',
    'CM':'horizon_layer',
    'Limite Superior': 'upper_depth',
    'Limite inferior': 'lower_depth',
    'Grau de humidade': 'moisture_degree',
    'Quantidade de raízes': 'root_quantity',
    'Diâmetro de raízes': 'root_diameter',
    'Manchas': 'stains',
    'Textura': 'texture',
    'Abundância de elementos grosseiros': 'thick_contents_count',
    'Forma de elementos grosseiros': 'thick_contents_shape',
    'Natureza de elementos grosseiros': 'thick_contents_nature',
    'Tipo de estrutura': 'structure_type',
    'Classes de estrutura': 'structure_class',
    'Grau de estrutura': 'structure_degree',
    'Diâmetro de poros': 'pore_diameter',
    'Quantidade de poros': 'pore_quantity',
    'Forma de poros': 'pore_shape',
    'Cor (s)': 'dry_color_name',
    'Matiz (s)': 'dry_hue',
    'Valor (s)':'dry_value',
    'Croma (s)': 'dry_chroma',
    'Cor (h)': 'moist_color_name',
    'Matiz (h)': 'moist_hue',
    'Valor (h)': 'moist_value',
    'Croma (h)': 'moist_chroma',
    'Compacidade':'compaction',
    'Dureza': 'durability', 
    'Friabilidade': 'friability'
}, inplace=True)

# # Drop unnecessary columns
# morphology_cleaning = morphology.drop(columns=[
#     'ID1', 'Agrupamento', 'REF', 'Pro', 'Observaçoes', 'Horizonte de diagnóstico', 'Propriedade de diagnóstico', 'Nitidez do limite', 'Designação do horizonte', 'Observaçoes', 'Confirmar', 'Adesividade', 'Plasticidade', 'Efervescência com HCl', 'Friabilidade', 'Orientação das Fendas', 'Largura das fendas', 'Quantidade de fendas'
# ])

morphology_cleaning = morphology
#add profile_record_id to populate later from soil profile table
#morphology_cleaning['profile_record_id'] = pd.NA

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
morphology_cleaning = morphology_cleaning.applymap(remove_accents)

In [4]:
# Reorder columns to match SAMPLES schema
morphology_cleaning1 = morphology_cleaning [[
    'morpho_id',
    'sample_id',
    #'profile_record_id',
    'profile',
    'horizon_layer',
    'upper_depth',
    'lower_depth',
    'moisture_degree',
    'root_quantity',
    'root_diameter',
    'texture',
    'structure_type',
    'structure_class',
    'structure_degree',
    'pore_diameter',
    'pore_quantity',
    'pore_shape',
    'dry_color_name',
    'dry_hue',
    'dry_value',
    'dry_chroma',
    'moist_color_name',
    'moist_hue',
    'moist_value',
    'moist_chroma',
    'compaction',
    'durability',
    'friability',
    'thick_contents_count',
    'thick_contents_nature'
]]

# Show first few rows
morphology_cleaning1.head(5)

,morpho_id,sample_id,profile,horizon_layer,upper_depth,lower_depth,moisture_degree,root_quantity,root_diameter,texture,...,dry_chroma,moist_color_name,moist_hue,moist_value,moist_chroma,compaction,durability,friability,thick_contents_count,thick_contents_nature
0,B_101/62_1_1,10999.0,101/62,1.0,0.0,11.0,Seco,Muitas finas e bastantes medias,NaN,Arenoso,...,2.5,Pardo-acinzentado-escuro,10YR,4.0,2.0,Pequena a minima,Brando,NaN,NaN,NaN
1,B_101/62_2_1,11000.0,101/62,2.0,11.0,28.0,Seco,Bastantes finas e medias e raras grossas,NaN,Arenoso-franco,...,3.0,Pardo-amarelado-escuro,10YR,3.0,4.0,Pequena,Brando,NaN,NaN,NaN
2,B_101/62_3_1,11001.0,101/62,3.0,28.0,54.0,Seco,Algumas finas e medias e raras grossas,NaN,Arenoso-franco,...,4.0,Pardo-amarelado-escuro,10YR,4.0,4.0,Pequena a minima,Brando,NaN,NaN,NaN
3,B_101/62_4_1,11002.0,101/62,4.0,54.0,90.0,Seco,"Poucas finas, algumas medias e raras grossas",NaN,Franco-arenoso a arenoso-franco,...,6.0,Pardo-forte,"7,5YR",5.0,6.0,Pequena a minima,Brando,NaN,NaN,NaN
4,B_101/62_5_2,11003.2,101/62,5.0,90.0,160.0,Seco a humido,Raras,Medias e grossas,Arenoso-franco,...,6.0,Pardo-forte,"7,5YR",5.0,6.0,Pequena,Brando,NaN,NaN,NaN


In [5]:
# Ensure consistent data types and formatting
morphology_cleaning1['profile'] = morphology_cleaning1['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]

#convert sample_to to string of the integer if it's a float like 11003.0
#The original string if it's not an integer or has decimal places
#"NULL" if the value is missing
def clean_sample_id(x):
    if pd.isnull(x):
        return "NULL"
    try:
        float_val = float(x)
        if float_val.is_integer():
            return str(int(float_val))
        else:
            return str(float_val)
    except:
        return str(x)

morphology_cleaning1['sample_id'] = morphology_cleaning1['sample_id'].apply(clean_sample_id)


/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_11715/4253896047.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morphology_cleaning1['profile'] = morphology_cleaning1['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]
/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_11715/4253896047.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morphology_cleaning1['sample_id'] = morphology_cleaning1['sample_id'].apply(clean_sample_id)


In [13]:
# Check for nulls in 'sample_id' column
null_string_values = morphology_cleaning1[morphology_cleaning1['sample_id'] == "NULL"]['sample_id'].unique()
print(null_string_values)

null_string_count = (morphology_cleaning1['sample_id'] == "NULL").sum()
print(f"🔍 Number of sample_id values equal to 'NULL': {null_string_count}")

#check for nulls in morpho_id column
null_string_values = morphology_cleaning1[morphology_cleaning1['morpho_id'] == "NULL"]['sample_id'].unique()
print(null_string_values)

null_string_count = (morphology_cleaning1['morpho_id'] == "NULL").sum()
print(f"🔍 Number of samorpho_idmple_id values equal to 'NULL': {null_string_count}")

['NULL']
🔍 Number of sample_id values equal to 'NULL': 6638
[]
🔍 Number of samorpho_idmple_id values equal to 'NULL': 0


**Goal** = drop any morpho_id from morphology_cleaning that does not have both:

1. Key horizon characteristics

2. A valid profile (w location and site characteristics) that appears in site_info_clean

In [6]:
import pandas as pd

# -----------------------------------------
# 1. Standardize 'profile' column formatting
# -----------------------------------------
site_info_clean.loc[:, 'profile'] = (
    site_info_clean['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str.upper()
    .str[:20]
)

morphology_cleaning1.loc[:, 'profile'] = (
    morphology_cleaning1['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str.upper()
    .str[:20]
)

# ---------------------------
# 2. Define valid profiles
# ---------------------------
valid_profiles = set(site_info_clean['profile'])

# ---------------------------
# 3. Define helper to check for missing/zero
# ---------------------------
def is_missing(val):
    return pd.isna(val) or val == 0 or val == '0'

# ---------------------------
# 4. Build masks for filtering
# ---------------------------

# A. Missing key horizon data
missing_key_data_mask = (
    morphology_cleaning1['morpho_id'].notna() &
    morphology_cleaning1['moist_color_name'].apply(is_missing) &
    morphology_cleaning1['texture'].apply(is_missing) &
    morphology_cleaning1['structure_type'].apply(is_missing)
)

# B. Invalid profile
invalid_profile_mask = ~morphology_cleaning1['profile'].isin(valid_profiles)

# C. Missing both upper and lower depth
missing_depth_mask = (
    morphology_cleaning1['upper_depth'].apply(is_missing) &
    morphology_cleaning1['lower_depth'].apply(is_missing)
)

# ---------------------------
# 5. Combine all masks
# ---------------------------
rows_to_flag_mask = missing_key_data_mask | invalid_profile_mask | missing_depth_mask
rows_to_flag = morphology_cleaning[rows_to_flag_mask]

# ---------------------------
# 6. Report and drop flagged rows
# ---------------------------
print(f"⚠️ Found {len(rows_to_flag)} rows with at least one of the following issues:")
print("   • missing key horizon data")
print("   • invalid profile (not in site_info_clean)")
print("   • missing both upper_depth and lower_depth")

# Optional: Save flagged rows for review
# rows_to_flag.to_csv("/Users/inesschwartz/Desktop/morph_flagged_rows_combined.csv", index=False)

# Drop flagged rows from main DataFrame
morphology_cleaning1 = morphology_cleaning1[~rows_to_flag_mask].copy()

# Summary after cleanup
print(f"✅ Remaining rows in morphology_cleaning: {len(morphology_cleaning1)}")


⚠️ Found 2597 rows with at least one of the following issues:
   • missing key horizon data
   • invalid profile (not in site_info_clean)
   • missing both upper_depth and lower_depth
✅ Remaining rows in morphology_cleaning: 10438


In [7]:
# check for duplicates
# Make sure sample_id and horizon_id are strings
morphology_cleaning1['sample_id'] = morphology_cleaning1['sample_id'].astype(str).str.strip()
morphology_cleaning1['morpho_id'] = morphology_cleaning1['morpho_id'].astype(str).str.strip()
# Filter out and print duplicated samples (including the first occurrence)
duplicated_values = morphology_cleaning1['sample_id'][morphology_cleaning1['sample_id'].duplicated()].unique()
print(duplicated_values)

['11016' '11017' 'NULL' '11352' '11353' '17532' '17533' '17534' '2438'
 '7192' '13178' '20883' '9027' '9231' '8232' '9255' '8639' '8536' '4918']


In [8]:
#addressing duplicates

# -----------------------------------
# 1. Correct sample_id using morpho_id map
# -----------------------------------
# Step 1: Make sure IDs are strings and trimmed
morphology_cleaning1['morpho_id'] = morphology_cleaning1['morpho_id'].astype(str).str.strip()
morphology_cleaning1['sample_id'] = morphology_cleaning1['sample_id'].astype(str).str.strip()

# Step 2: Your mapping (morpho_id → new sample_id)
horizon_to_sample_map = {
    'UZ_24c/60_1_1': '8639',
    'UZ_24c/60_2_1': '8640',
    'UZ_24c/60_3_1': '8641',
    'UZ_24c/60_4_1': '8642',
    'UZ_24c/60_5_1': '8643',
    'UZ_66c/60_1_1': '8777',
    'UZ_66c/60_2_1': '8778',
    'UZ_66c/60_3_1': '8779',
    'UZ_66c/60_4_1': '8780',
    'CC_473/66_1_1': '17596',
    'CC_473/66_2_1': '17597',
    'CC_473/66_3_1': '17598',
    'N_85/57_1_0': '4947',
    'N_85/57_2_0': '4948',
    'N_85/57_3_0': '4949',
    'UZ_231/60_4_1': '8255',
    'B_139/61_5_1': '10353',
    'H_480/55_1_1': '3919',
    'H_91/54_1_1': '1255',
    'H_90/54_3_3': '1252',
    'Hb_136/56_1_1':'2961',
    'Hb_136/56_2_1': '2962',
    'Hb_136/56_3_1': '2963',
    'Hb_136/56_4_1': '2964',
    'UZ_216/60_5_1': '8231',
    'Bg_217/46_1_1': '694',
    'Bg_217/46_2_1': '695',
    'Bg_217/46_3_1': '696',
    'UZ_215c/60_1_1':'9255',
    'UZ_215c/60_2_1': '9256',
    'UZ_215c/60_3_1': '9257',
    'UZ_215c/60_4_1': '9258',
    'UZ_215c/60_5_1': '9259',
    'UZ_215c/60_6_1': '9260',
    'N_30/57_1_1':'4848',
    'N_30/57_2_2':'4849',
    'N_30/57_3_1':'4850',
    'UZ_66c/60_1_1': '8777',
    'UZ_66c/60_2_1': '8778',
    'UZ_66c/60_3_1': '8779',
    'UZ_66c/60_4_1': '8780',
    'UZ_66c/60_5_1': '8781',
    'UZ_66c/60_6_1': '8783',
    'UZ_66c/60_7_1': '8782',
    'H_91/54_1_1': '1255',
    'H_91/54_2_1': '1256',
    'H_91/54_3_1': '1257'
}

# Step 3: Apply the mapping to replace sample_id based on horizon_id
morphology_cleaning1['sample_id'] = morphology_cleaning1.apply(
    lambda row: horizon_to_sample_map.get(row['morpho_id'], row['sample_id']),
    axis=1
)

# -----------------------------------
# 2. Drop specific horizon_ids (might have already been dropped)
# -----------------------------------
horizons_to_drop = [
    'B_139/61_4_1', 'H_113/55_1_1', 'H_1306/52_1_1', 'H_1611/52_1_1',
    'H_227/46_1_1', 'H_227/46_2_1', 'H_227/46_3_1', 'H_227/46_4_1',
    'Mj_178c/63_4_1', 'Mj_321c/63_4_1', 'B_110/62_5_1', 'B_110/62_6_1'
]
morphology_cleaning1 = morphology_cleaning1[~morphology_cleaning1['morpho_id'].isin(horizons_to_drop)]


# -----------------------------------
# 3. Done — Optional: Save to file
# -----------------------------------
#df_morpho_cleaned.to_csv("/Users/inesschwartz/Desktop/df_morpho_cleaned_corrected.csv", index=False)
print("✅ Morphology data cleaned and sample_id updated via morpho_id.")


✅ Morphology data cleaned and sample_id updated via morpho_id.


In [9]:
#re-count null sample_id's
morphology_cleaning1
# Check for nulls in 'sample_id' column
null_string_values2 = morphology_cleaning1[morphology_cleaning1['sample_id'] == "NULL"]['sample_id'].unique()
print(null_string_values2)

null_string_count2 = (morphology_cleaning1['sample_id'] == "NULL").sum()
print(f"🔍 Number of sample_id values equal to 'NULL': {null_string_count2}")

['NULL']
🔍 Number of sample_id values equal to 'NULL': 4713


In [10]:
num_duplicates = morphology_cleaning1['morpho_id'].duplicated().sum()
print(f"🔁 Found {num_duplicates} duplicated morpho_id(s).")


🔁 Found 0 duplicated morpho_id(s).


In [11]:
df_morpho_cleaned = morphology_cleaning1

### dealing w decimals in sample_id

In [12]:
# Step 1: Ensure sample_id is a string
df_morpho_cleaned['sample_id'] = df_morpho_cleaned['sample_id'].astype(str).str.strip()

# Step 2: Identify sample_ids that contain a decimal point
decimal_sample_ids = df_morpho_cleaned[df_morpho_cleaned['sample_id'].str.contains(r'\.\d+$', regex=True)]

# Step 3: Output results
print(f"🔍 Found {len(decimal_sample_ids)} sample_id(s) with decimal values.\n")

# Optional: Print the unique sample_ids with decimals
print("🧾 Affected sample_ids:")
print(decimal_sample_ids['sample_id'].unique())


🔍 Found 439 sample_id(s) with decimal values.

🧾 Affected sample_ids:
['11003.2' '11285.2' '10843.2' '10428.2' '10470.2' '10480.2' '10485.2'
 '10446.2' '10849.2' '10509.2' '10562.2' '10536.2' '11327.2' '11356.2'
 '11393.2' '11176.2' '11454.2' '10885.2' '10904.2' '10922.2' '11485.2'
 '10933.2' '4972.3' '4981.2' '5810.2' '5045.2' '5822.2' '4989.2' '5006.2'
 '5010.2' '5839.2' '4996.2' '5082.2' '5026.2' '6314.2' '6317.2' '6323.2'
 '6332.2' '6336.3' '5883.2' '6342.3' '6348.2' '6350.2' '6358.2' '5902.2'
 '5897.3' '6376.3' '6381.2' '6383.3' '6388.2' '6391.2' '6402.2' '6405.2'
 '6416.2' '5914.2' '5916.2' '6439.3' '5088.2' '5053.2' '5101.2' '5107.2'
 '5123.2' '5945.2' '5174.2' '5176.2' '5963.2' '5973.2' '5202.2' '5207.2'
 '5192.2' '5214.2' '5218.2' '5225.3' '5983.2' '5990.2' '5270.2' '5995.2'
 '5997.2' '5447.2' '6014.2' '6022.2' '5417.2' '5332.2' '6032.2' '5476.2'
 '5478.2' '5434.2' '5359.2' '5442.2' '5444.2' '5327.2' '5342.2' '5315.2'
 '5348.2' '7390.2' '7397.2' '5717.2' '5745.2' '5751.2' '573

In [13]:
## take out all decimals in sample_id (keep as just whole number)
# Function to truncate decimal part and return string
def truncate_to_int_string(x):
    try:
        return str(int(float(x)))
    except:
        return str(x).strip()  # fallback to string version if it can't convert

# Apply to sample_id column
df_morpho_cleaned['sample_id'] = df_morpho_cleaned['sample_id'].apply(truncate_to_int_string)

In [22]:
# save to csv
df_morpho_cleaned.to_csv("/Users/inesschwartz/Desktop/morpho_cleaned_updated.csv", index=False)

# Lab_Info  table

In [2]:
analyses = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Horizontes Analises.xlsx")

analyses.columns

Index(['Amostra', 'ID', 'Morfo_id', 'Analise-id', 'PERFIL', 'GR', 'COD_PROV',
       'NA', 'LS', 'LI', 'EG', 'AG', 'AF', 'L', 'Argila', 'Eq_Hum', 'atm_1/3',
       'atm_15', 'CACO3', 'Gesso', 'Fe livre', 'CO', 'N total', 'C/N', 'MO',
       'P205 total', 'pH (H2O)', 'pH(KCL)', 'Ca++', 'Mg++', 'K+', 'Na+',
       'Soma de bases', 'CTC', 'V', 'Cloretos', 'Sulfatos', 'Condutividade',
       'Sódio solúvel', 'EqMol (SiO2)', 'EqMol(Al2O3)', 'EqMol(Fe2O3)',
       'SiO2/Al2O3', 'SiO2/Fe2O3', 'SiO2/R2O3', 'Fe2O3/Al2O3', 'FE2O3_TARG',
       'FE2O3_LARG', 'CEC_ARG', 'Min_<0,002', 'Min_0,05-0,02', 'Min_0,2-0,05',
       'Min_2-0,2', 'Confirmar'],
      dtype='object')

In [5]:
#change column names
analyses.rename(columns={
    'Amostra': 'sample_id',
    'Morfo_id': 'morpho_id',
    'Analise-id': 'analysis_id',
    'PERFIL': 'profile', #change to reference PK profile_record_id
    'LS': 'upper_limit',
    'LI': 'lower_limit',
    'EG': 'EG',
    'AG': 'thick_sand',
    'AF': 'fine_sand',
    'Argila': 'clay',
    'L': 'silt',
    'Gesso': 'gypsum',
    'Fe livre': 'free_iron',
    'CO':'organic_carbon',
    'N total': 'total_N',
    'MO': 'OM',
    'Soma de bases': 'exchangable_bases_sum',
    'CTC': 'CEC',
    'Cloretos':'chlorides',
    'Sulfatos':'sulfates',
    'Condutividade':'conductivity',
    'Sódio solúvel':'soluble_sodium',
    'P205 total': 'P205',
    'pH (H2O)':'pH_H2O',
    'pH(KCL)':'pH_KCL',
    'MO': 'organic_material',
    'atm_1/3': 'atm_1_3',
    'Ca++': 'Ca',
    'Mg++': 'Mg',
    'Na+': 'Na',
    'K+': 'K',
    'Min_<0,002': 'Min_lt_0002',
    'Min_0,05-0,02': 'Min_005_002',
    'Min_0,2-0,05': 'Min_02_005',
    'Min_2-0,2': 'Min_2_02',
}, inplace=True)

# Drop columns not needed
analyses_drop = analyses.drop([
    'EqMol (SiO2)', 'EqMol(Al2O3)', 'EqMol(Fe2O3)', 'SiO2/Al2O3', 
    'SiO2/Fe2O3', 'SiO2/R2O3', 'Fe2O3/Al2O3', 'FE2O3_TARG', 
    'FE2O3_LARG', 'CEC_ARG', 'GR', 'COD_PROV'
], axis=1)

In [6]:
# Add a new Primary Key ID column starting from 1
#drop lab_sample_id if already exist error comes up
# Check if 'lab_sample_id' exists and drop it
if 'lab_sample_id' in analyses_drop.columns:
    analyses_drop = analyses_drop.drop(columns=['lab_sample_id'])

# Insert a new column, e.g., 'lab_sample_id' as a primary key starting from 1
analyses_drop.insert(0, 'lab_sample_id', range(1, len(analyses_drop) + 1))

#add minerology_id column
analyses_drop['minerology_id'] = pd.NA
#add soil_biology_id column (just in case)
analyses_drop['soil_biology_id'] = pd.NA


#Define the desired final column order
final_columns = [
    'lab_sample_id',
    'analysis_id',
    'morpho_id',
    'sample_id',
    'profile',
    'minerology_id',
    'soil_biology_id',
    'upper_limit',
    'lower_limit',
    'EG',
    'thick_sand',
    'fine_sand',
    'silt',
    'clay',
    'Eq_Hum',
    'atm_1_3',
    'atm_15',
    'CACO3',
    'gypsum',
    'free_iron',
    'organic_carbon',
    'total_N',
    'P205',
    'organic_material',
    'pH_H2O',
    'pH_KCL',
    'Ca',
    'Mg',
    'Na',
    'K',
    'exchangable_bases_sum',
    'CEC',
    'V',
    'conductivity',
    'soluble_sodium',
    'Min_lt_0002',
    'Min_005_002',
    'Min_02_005',
    'Min_2_02',
]

analyses1 = analyses_drop[final_columns]

In [7]:
#address nulls 
# Show count of NaN values in each column
nan_counts = analyses1.isna().sum()

# Filter to show only columns with at least one NaN
nan_columns = nan_counts[nan_counts > 0]

# Print them
print("🔍 Columns with NaN values:")
print(nan_columns)

#formatting and checking for duplicates
# Make a safe copy
analyses1 = analyses_drop[final_columns].copy()

# Convert sample_id to numeric, coercing errors to NaN
analyses1['sample_id'] = pd.to_numeric(analyses1['sample_id'], errors='coerce')

# Drop rows with NaN in sample_id to avoid int conversion error
analyses1 = analyses1.dropna(subset=['sample_id'])

# Convert to int, then to str
analyses1['sample_id'] = analyses1['sample_id'].astype(int).astype(str)

# Clean the profile column
analyses1['profile'] = (
    analyses1['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str[:20]
)

# Alias to another variable
analyses2 = analyses1

🔍 Columns with NaN values:
sample_id                  19
minerology_id            7847
soil_biology_id          7847
upper_limit                 7
lower_limit                 7
EG                       4910
thick_sand                448
fine_sand                 390
silt                      430
clay                      420
Eq_Hum                   2297
atm_1_3                  7469
atm_15                   5689
CACO3                    7334
gypsum                   7815
free_iron                2792
organic_carbon           2395
total_N                  6259
P205                     5464
organic_material         2413
pH_H2O                    684
pH_KCL                   1749
Ca                       3018
Mg                       3025
Na                       3024
K                        2995
exchangable_bases_sum    6740
CEC                      2423
V                        2476
conductivity             7201
soluble_sodium           1270
Min_lt_0002              7517
Min_005_002  

In [9]:
# Filter out and print duplicated samples (including the first occurrence)
duplicated_values = analyses2['sample_id'][analyses2['sample_id'].duplicated()].unique()
print(duplicated_values)

# 1. Ensure sample_id and analysis_id are strings
analyses2['sample_id'] = analyses2['sample_id'].astype(str).str.strip()
analyses2['analysis_id'] = analyses2['analysis_id'].astype(str).str.strip()

# 2. Correct duplicates for sample_id '11608'
mask_11608 = analyses2['sample_id'] == '11608'
analyses2.loc[mask_11608, 'sample_id'] = [
    f"11608_{i+1}" for i in range(mask_11608.sum())
]

# 3. Remove duplicate for sample_id '3497' (keep first occurrence only)
duplicates_3497 = analyses2[analyses2['sample_id'] == '3497']
if len(duplicates_3497) > 1:
    indices_to_drop = duplicates_3497.index[1:]
    analyses2 = analyses2.drop(indices_to_drop)

# 4. Correct sample_id → analysis_id assignments for specific sample_ids
id_map = {
    'UZ_24c/60_1_1': '8639',
    'UZ_24c/60_2_1': '8640',
    'UZ_24c/60_3_1': '8641',
    'UZ_24c/60_4_1': '8642',
    'UZ_24c/60_5_1': '8643'
}

# Apply mapping to update analysis_id based on sample_id
analyses2['analysis_id'] = analyses2.apply(
    lambda row: id_map.get(row['sample_id'], row['analysis_id']),
    axis=1
)

print("✅ Sample ID corrections applied.")


['10337' '3498' '2923' '2924' '2925' '2926' '2927' '2928' '2929' '2930'
 '16355' '16356' '419' '420' '15682' '13817' '75' '13965' '3418' '9230'
 '8639' '3529']
✅ Sample ID corrections applied.


In [10]:
#drop unwanted/un-needed duplicates or erroneous reccords

# Drop second row where sample_id == '3497'
indices = analyses2.index[analyses2['sample_id'] == '3497'].tolist()
if len(indices) > 1:
    analyses2 = analyses2.drop(indices[1])

# Drop second row where sample_id == '3497'
indices = analyses2.index[analyses2['sample_id'] == '13817'].tolist()
if len(indices) > 1:
    analyses2 = analyses2.drop(indices[1])

# drop second row where sample_id == '8639'
indices = analyses2.index[analyses2['sample_id'] == '8639'].tolist()
if len(indices) > 1:
    analyses2 = analyses2.drop(indices[1])

#Drop second row where sample_id =='2923-2930'
# Ensure sample_id is string for consistent comparison
analyses2['sample_id'] = analyses2['sample_id'].astype(str)

# Define the sample_ids to clean duplicates for
target_ids = [str(i) for i in range(2923, 2931)]

# Identify and drop the second occurrence of each
for sid in target_ids:
    matches = analyses2.index[analyses2['sample_id'] == sid].tolist()
    if len(matches) > 1:
        # Drop the second occurrence (index 1 in the list)
        analyses2 = analyses2.drop(matches[1])
        print(f"🗑 Dropped second occurrence of sample_id {sid}")


# Ensure sample_id is string for correct matching
analyses2['sample_id'] = analyses2['sample_id'].astype(str)

# List of sample_ids to drop (as strings)
sample_ids_to_drop = [
    '4247', '4248', '4249', '4250', '4251', '4252', '4253', '4254',  # invalid samples
    '5533', '6636', '6637', '6638', '6639',
    '6875', '7361',
    '3012', '3013', '3014', '3015', '3016',
    '3049', '3050', '3051', '3052', '3053', '3054', '419', '420', '421', '422'
]

# Drop rows where sample_id is in the list
analyses2 = analyses2[~analyses2['sample_id'].isin(sample_ids_to_drop)]


🗑 Dropped second occurrence of sample_id 2923
🗑 Dropped second occurrence of sample_id 2924
🗑 Dropped second occurrence of sample_id 2925
🗑 Dropped second occurrence of sample_id 2926
🗑 Dropped second occurrence of sample_id 2927
🗑 Dropped second occurrence of sample_id 2928
🗑 Dropped second occurrence of sample_id 2929
🗑 Dropped second occurrence of sample_id 2930


In [11]:
# Ensure both 'lab_sample_id' and 'sample_id' columns exist
assert 'lab_sample_id' in analyses2.columns, "Missing column: lab_sample_id"
assert 'sample_id' in analyses2.columns, "Missing column: sample_id"

# Step 1: Ensure lab_sample_id is string so comparison works
analyses2['lab_sample_id'] = analyses2['lab_sample_id'].astype(str)

# Step 2: Manual corrections mapping
manual_corrections = {
    '66': 10377,
    '4219': 3497,
    '4221': 3498,   # same sample, different depths
    '4222': 3499,
    '4747': 16255,
    '4748': 16256,
    '5010': 15862,
    '6802': 13695,
    '7255': 8230,
    '7686': 8529,
    '6875': 3419,
    '6638': 75_1,
}

# Step 3: Apply manual corrections
for lab_id, sample_id in manual_corrections.items():
    matches = analyses2['lab_sample_id'] == lab_id
    if matches.sum() == 0:
        print(f"⚠️ Warning: lab_sample_id {lab_id} not found in data.")
    else:
        analyses2.loc[matches, 'sample_id'] = sample_id

# Step 4: Summary after correction
print("✅ Manual sample_id corrections applied.")
print(f"Remaining rows: {len(analyses2)}")
print(f"Missing sample_id entries: {analyses2['sample_id'].isna().sum()}")

# Optional: Show which rows are still missing sample_id
missing_sample_ids = analyses2[analyses2['sample_id'].isna()]
if not missing_sample_ids.empty:
    print("🔍 Rows with missing sample_id after correction:")
    print(missing_sample_ids[['lab_sample_id', 'sample_id']])


✅ Manual sample_id corrections applied.
Remaining rows: 7798
Missing sample_id entries: 0


In [12]:
# Find rows with missing sample_id
missing_sample_id_rows = analyses2[analyses2['sample_id'].isna()]

# Print how many are missing
print(f"❌ Missing sample_id in {len(missing_sample_id_rows)} rows.")

# Show the unique lab_sample_id values that are missing a sample_id
print("🔍 lab_sample_id values with missing sample_id:")
print(missing_sample_id_rows['lab_sample_id'].unique())

# Optional: Save to CSV for review
#missing_sample_id_rows.to_csv("missing_sample_ids.csv", index=False)


❌ Missing sample_id in 0 rows.
🔍 lab_sample_id values with missing sample_id:
[]


In [13]:
# Drop rows with missing sample_id (NaN only)
analyses2 = analyses2[analyses2['sample_id'].notna()].copy()

analyses3 = analyses2


In [14]:
# Filter out and print duplicated samples (including the first occurrence)
duplicated_values = analyses3['sample_id'][analyses3['sample_id'].duplicated()].unique()
print(duplicated_values)

[]


## joining elemental analyses to analyses


In [15]:
# Rename and clean sample_id as before
elemental_analyses.rename(columns={'Lab. Code': 'sample_id'}, inplace=True)

# Convert to string to clean text
sample_id_cleaned = elemental_analyses['sample_id'].astype(str)
sample_id_cleaned = sample_id_cleaned.str.replace('C-', '', regex=False)
sample_id_cleaned = sample_id_cleaned.str.replace(' MNL', '', regex=False)

# Convert to numeric with coercion (NaNs will appear here)
elemental_analyses['sample_id'] = pd.to_numeric(sample_id_cleaned, errors='coerce')

# 🔍 Show rows where sample_id is NaN
nan_sample_ids = elemental_analyses[elemental_analyses['sample_id'].isna()]
print(f"❌ Found {len(nan_sample_ids)} rows with NaN sample_id:")
print(nan_sample_ids)


❌ Found 0 rows with NaN sample_id:
Empty DataFrame
Columns: [sample_id, Soil Profile, Code, Code.1, Depht, Sampling Date, Mg, Al, Si, P, S, Cl, K, Ca, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn, As, Se, Rb, Sr, Zr, Nb, Mo, Cd, Sn, Sb, Ba, Ta, W, Pt, Au, Hg, Tl, Pb, Bi, Th, U]
Index: []

[0 rows x 44 columns]


In [16]:
# Rename 'Lab. Code' to 'sample_id'
elemental_analyses.rename(columns={'Lab. Code': 'sample_id'}, inplace=True)

# Temporarily convert to string to do the replacements
sample_id_cleaned = elemental_analyses['sample_id'].astype(str)
sample_id_cleaned = sample_id_cleaned.str.replace('C-', '', regex=False)
sample_id_cleaned = sample_id_cleaned.str.replace(' MNL', '', regex=False)

# Convert cleaned values to numeric safely
elemental_analyses['sample_id'] = pd.to_numeric(sample_id_cleaned, errors='coerce')


# Convert to integer and then string for uniformity
elemental_analyses['sample_id'] = elemental_analyses['sample_id'].astype(int).astype(str)

# Rename 'Code' to 'field_sample_code'
elemental_analyses.rename(columns={'Code': 'field_sample_code'}, inplace=True)


In [17]:
#Convert sample_id to string (no truncation unless necessary)
elemental_analyses['sample_id'] = elemental_analyses['sample_id'].astype(str).str.strip()


In [18]:
print (analyses3['sample_id'].dtype)
print (elemental_analyses['sample_id'].dtype)


object
object


In [19]:
# Check which sample_ids in elemental_analyses are NOT in analyses_ready
missing_ids = elemental_analyses[~elemental_analyses['sample_id'].isin(analyses3['sample_id'])]

# Display them
print("Sample IDs in elemental_analyses not found in analyses2:")
print(missing_ids['sample_id'].unique())

print(len(missing_ids['sample_id'].unique()))



Sample IDs in elemental_analyses not found in analyses2:
['16105' '16217' '16033' '16231' '16208' '16274' '16225' '15731' '16078'
 '15786' '15693' '15678' '15959' '16372' '16418' '16459' '16477' '15498'
 '15437' '12749' '13109' '14337' '14319' '15484' '15508' '15614' '15580'
 '15582' '15606' '15463' '14496' '11343' '17892' '16976' '17269' '18248'
 '16946' '17686' '17728' '17415' '17642' '18212' '17016' '17282' '18429'
 '17340' '18824' '18448' '8721' '7313' '5502' '5392' '7317' '8626' '7193'
 '7254' '7293' '8240' '15397' '14403' '14983' '16126' '15059' '15322'
 '15268' '14808' '15106']
67


In [20]:
# LEFT JOIN ELEMENTAL ANALYSE TO ANALYSE TABLE

merged1 = analyses3.merge(elemental_analyses, on='sample_id', how='left')
merged1.head()


,lab_sample_id,analysis_id,morpho_id,sample_id,profile,minerology_id,soil_biology_id,upper_limit,lower_limit,EG,...,Ta,W,Pt,Au,Hg,Tl,Pb,Bi,Th,U
0,1,B_101/62_1_1,B_101/62_1_1,10999,101_62,<NA>,<NA>,0.0,11.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,B_101/62_2_1,B_101/62_2_1,11000,101_62,<NA>,<NA>,11.0,28.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,B_101/62_3_1,B_101/62_3_1,11001,101_62,<NA>,<NA>,28.0,54.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,B_101/62_4_1,B_101/62_4_1,11002,101_62,<NA>,<NA>,55.0,85.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,B_101/62_5_21,B_101/62_5_2,11003,101_62,<NA>,<NA>,90.0,120.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
print(analyses3[analyses3['sample_id'] == '6940'])
print(elemental_analyses[elemental_analyses['sample_id'] == '6940'])


     lab_sample_id    analysis_id      morpho_id sample_id profile  \
1361          1362  Cb_134/59_6_1  Cb_134/59_6_1      6940  134_59   

     minerology_id soil_biology_id  upper_limit  lower_limit  EG  ...   K  \
1361          <NA>            <NA>        130.0        170.0 NaN  ... NaN   

      exchangable_bases_sum  CEC   V  conductivity  soluble_sodium  \
1361                    NaN  NaN NaN           NaN             0.0   

      Min_lt_0002  Min_005_002  Min_02_005  Min_2_02  
1361          NaN          NaN         NaN       NaN  

[1 rows x 39 columns]
   sample_id Soil Profile field_sample_code Code.1      Depht Sampling Date  \
80      6940      P134/59          A-320/59  S-492  1.30-1.70    1954-08-06   

             Mg             Al             Si      P  ...    Ta   W  Pt  Au  \
80  1457.666667  106266.333333  308859.666667  468.0  ...  26.0 NaN NaN NaN   

           Hg   Tl         Pb  Bi         Th   U  
80  10.666667  4.0  25.333333 NaN  12.333333 NaN  

[1 rows x

In [22]:
# Define final column list
final_columns = [
    'lab_sample_id', 'analysis_id', 'morpho_id','sample_id', 'profile', 'soil_biology_id', 'upper_limit', 'lower_limit', 'EG', 'thick_sand', 'fine_sand', 'silt', 'clay', 'Eq_Hum', 'atm_1/3', 'atm_15',
    'CACO3', 'gypsum', 'free_iron', 'organic_carbon', 'total_N', 'P205', 'organic_material', 'pH_H2O', 'pH_KCL',
    'Ca++', 'Mg++', 'Na+', 'K+', 'exchangable_bases_sum', 'CEC', 'V', 'conductivity', 'soluble_sodium', 'Min_<0,002',
    'Min_0,05-0,02', 'Min_0,2-0,05', 'Min_2-0,2', 
    'field_sample_code', 'Depth', 'Al', 'Si', 'P', 'S', 'Cl', 'Ti', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'As', 'Se', 'Rb', 'Sr', 'Zr', 'Nb', 'Mo', 'Cd', 'Sn', 'Sb', 'Ba', 'Ta', 'W', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi',
    'Th', 'U',
]

# Ensure all columns exist in the DataFrame
for col in final_columns:
    if col not in merged1.columns:
        merged1[col] = pd.NA  # or np.nan if preferred

# Reorder DataFrame columns
merged1 = merged1[final_columns]

# Check and print the datatypes of each column
print("Data types of each column in merged1:\n")
print(merged1.dtypes)

Data types of each column in merged1:

lab_sample_id     object
analysis_id       object
morpho_id         object
sample_id         object
profile           object
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 74, dtype: object


In [23]:
# CHECK/CHANGE DATATYPES

# Change PK (sample_id) to a consistent datatype: Converted to string and truncate to 20 characters (similar to VARCHAR(20))
merged1['sample_id'] = merged1['sample_id'].astype(str).str.strip().str[:20]
merged1['lab_sample_id'] = merged1['lab_sample_id'].astype(str).str.strip().str[:20]

print(merged1.dtypes)

# CHECK AND/OR ADD FK
# Check if 'lab_sample_id' exists and drop it
if 'lab_sample_id' in merged1.columns:
    merged1 = merged1.drop(columns=['lab_sample_id'])

# Insert a new column, e.g., 'lab_sample_id' as a primary key starting from 1
merged1.insert(0, 'lab_sample_id', range(1, len(merged1) + 1))

lab_sample_id     object
analysis_id       object
morpho_id         object
sample_id         object
profile           object
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 74, dtype: object


In [24]:
# SAVE TO CSV
merged1.to_csv("/Users/inesschwartz/Desktop/analyses_clean.csv", index=False)

# Soil Type Table

In [54]:
# Rename columns
soil_type = soil_profile.copy()

soil_type.rename(columns={
    'Perfil': 'profile',
    'Agrupamento': 'grouping',
    'Pro': 'province',
    'País': 'country',
    'Local': 'location',
    'DATA': 'date',
    'CEP_NOME': 'CEP_NAME',
}, inplace=True)

# Drop unnecessary columns
soil_type_cleaning = soil_type.drop(columns=[
    'REF', 'province', 'country', 'location', 'DESCRITOR1', 'DESCRITOR2', 'DESCRITOR3',
    'date', 'Fase', 'D_INSERÇAO', 'Publicação', 'WRB_old', 'Missão'
], errors='ignore')  # use errors='ignore' in case some columns were already missing

# Add a new Primary Key ID column starting from 1
soil_type_cleaning.insert(0, 'soil_type_id', range(1, len(soil_type_cleaning) + 1))

# Drop accents
import unicodedata

def remove_accents(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in text if not unicodedata.combining(c))
    return text

# Apply to all cells in the DataFrame
soil_type_cleaning = soil_type_cleaning.applymap(remove_accents)

# Replace / with _ and strip/shorten 'profile' BEFORE slicing into site_info_clean
soil_type_cleaning['profile'] = (
    soil_type_cleaning['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str[:20]
)

# Keep only relevant columns
soil_type_clean = soil_type_cleaning[[
    'soil_type_id',
    'profile',
    'CEP_GR',
    'CEP_NAME',
    'FAO'
]]

# Preview
soil_type_clean.head(2)


,soil_type_id,profile,CEP_GR,CEP_NAME,FAO
0,1,1_51,NaN,NaN,NaN
1,2,1_57,Aridicos,Aridicos com calcario Pardo-cinzentos,CLha


In [55]:
# check duplicates
# Filter out and print duplicated samples (including the first occurrence)
duplicated_values_st = soil_type_clean['profile'][soil_type_clean['profile'].duplicated()].unique()
print(duplicated_values_st)

# Check for profile values that are the string "NULL"
null_string_values_st = soil_type_clean[soil_type_clean['profile'] == "NULL"]['profile'].unique()
print("🔍 Unique 'profile' values equal to string 'NULL':", null_string_values_st)

null_string_count_st = (soil_type_clean['profile'] == "NULL").sum()
print(f"🔢 Number of 'profile' values equal to string 'NULL': {null_string_count_st}")



[]
🔍 Unique 'profile' values equal to string 'NULL': []
🔢 Number of 'profile' values equal to string 'NULL': 0


In [56]:
# save clean table
soil_type_clean.to_csv("/Users/inesschwartz/Desktop/soil_type_clean.csv", index=False)

# Climate table


In [61]:
# Load original Excel data
profile_loc = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Perfis_local.xlsx")

# Create a copy for cleaning
climate_features_cleaning = profile_loc.copy()

# Mapping from climate codes to descriptions
code_to_description = {
    "B1": "Húmido",
    "B2": "Húmido",
    "B3": "Húmido",
    "B4": "Húmido",
    "C1": "Sub-húmido seco",
    "C2": "Sub-húmido chuvoso",
    "D": "Semi-árido",
    "E": "Árido",
    "Aw": "Tropical chuvoso com estação seca no Inverno, de savana",
    "BSw": "Seco de estepe, com chuva predominante no Verão",
    "BWw": "Seco de deserto, com chuva predominante no Verão",
    "Cw": "Mesotérmico húmido com estação seca no Inverno",
    "ARe": "Arídico extremo",
    "ARf": "Arídico fraco",
    "ARt": "Arídico típico",
    "tUDs": "Tempúdico seco",
    "tUSh": "Tempústico húmido",
    "tUSt": "Tempústico típico",
    "TUDs": "Tropúdico seco",
    "TUSa": "Tropústico arídico",
    "TUSu": "Tropústico údico",
    "TUSt": "Tropustico típico",
    "H": "Hipertérmico",
    "iH": "Iso-Hipertérmico",
    "iT": "Iso-Térmico",
    "T": "Térmico"
}

# Replace climate codes with descriptions
for col in ["CL_THORNTH", "CL_KOPPEN", "REG_HÍDRIC", "REG_TÉRMIC"]:
    climate_features_cleaning[col] = climate_features_cleaning[col].replace(code_to_description)

# Function to average values like "21-22" -> 21.5
def average_range(value):
    if isinstance(value, str) and '-' in value:
        try:
            nums = [float(x.strip()) for x in value.split('-')]
            return sum(nums) / len(nums)
        except:
            return None
    try:
        return float(value)
    except:
        return None

# Apply to temperature and precipitation columns
climate_features_cleaning["TMA"] = climate_features_cleaning["TMA"].apply(average_range)
climate_features_cleaning["PMA"] = climate_features_cleaning["PMA"].apply(average_range)

# Rename columns for clarity
climate_features_cleaning.rename(columns={
    'PERFIL': 'profile',
    'CL_THORNTH': 'thornthwaite_climate',
    'CL_KOPPEN': 'koppen_climate',
    'TMA': 'mean_annual_temp',
    'PMA': 'mean_annual_precip',
    'REG_HÍDRIC': 'hydric_regime',
    'REG_TÉRMIC': 'thermal_regime'
}, inplace=True)

# ensure consistent profile formatting
climate_features_cleaning['profile'] = climate_features_cleaning['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]

# Drop accents from text values
def remove_accents(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in text if not unicodedata.combining(c))
    return text

climate_features_cleaning = climate_features_cleaning.applymap(remove_accents)

# Add primary key column
climate_features_cleaning.insert(0, 'climate_id', range(1, len(climate_features_cleaning) + 1))

# Replace empty strings or nulls in numeric columns with \N (Postgres null)
for col in ['mean_annual_temp', 'mean_annual_precip']:
    climate_features_cleaning[col] = climate_features_cleaning[col].replace(
        ['', ' ', 'NULL', None, pd.NA, pd.NaT, 'nan', float('nan')], ''
    )

# Select and reorder relevant columns for export
climate_features_clean = climate_features_cleaning[[
    'climate_id',
    #'profile',
    'ID',
    'mean_annual_temp',
    'mean_annual_precip',
    'koppen_climate',
    'thornthwaite_climate',
    'hydric_regime',
    'thermal_regime'
]]

# Preview
climate_features_clean.head()


,climate_id,ID,mean_annual_temp,mean_annual_precip,koppen_climate,thornthwaite_climate,hydric_regime,thermal_regime
0,1,2770,,,NaN,Arido,NaN,NaN
1,2,48,,,NaN,Humido,NaN,NaN
2,3,1618,,,NaN,NaN,NaN,NaN
3,4,881,21.5,1500.0,"Tropical chuvoso com estacao seca no Inverno, ...",Humido,Tropustico udico,Iso-Hipertermico
4,5,1750,,,NaN,NaN,NaN,NaN


Check FK relationships and datatypes

In [63]:
# Check and print the datatypes of each column
print("Data types of each column in climate_clean:\n")
print(climate_features_clean.dtypes)

Data types of each column in climate_clean:

climate_id               int64
ID                       int64
mean_annual_temp        object
mean_annual_precip      object
koppen_climate          object
thornthwaite_climate    object
hydric_regime           object
thermal_regime          object
dtype: object


In [64]:
# save clean csv table
climate_features_clean.to_csv("/Users/inesschwartz/Desktop/climate_clean.csv", index=False)

# Topo features table

In [66]:
# Create a copy for cleaning
topo_features_cleaning = profile_loc.copy()

# Rename columns for clarity
topo_features_cleaning.rename(columns={
    'PERFIL': 'profile',
    'TOPOGRAFIA': 'slope_code',
    'ALTITUDE': 'altitude',
}, inplace=True)

# ensure consistent profile formatting
topo_features_cleaning['profile'] = topo_features_cleaning['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]

# Add missing columns
topo_features_cleaning['aspect'] = pd.NA  
topo_features_cleaning['land_surface_temp'] = pd.NA
topo_features_cleaning['dem_elevation'] = pd.NA  

# Add primary key column
topo_features_cleaning.insert(0, 'topo_id', range(1, len(topo_features_cleaning) + 1))

# Create slope class mapping dictionary
slope_code_to_description = {
    "D1": "Plano (Declives < 2%)",
    "D2": "Ondulado muito suave (Declives > 2% e < 3%)",
    "D3": "Ondulado suave (Declives > 3% e < 5%)",
    "D4": "Ondulado (Declives > 5% e < 8%)",
    "D5": "Acidentado (Declives > 8% e < 15%)",
    "D6": "Escarpado (Declives >15% e < 30%)",
    "D7": "Montanhoso (Declives > 30%)"
}

# Create a mapping DataFrame for slope classes
slope_classes_df = pd.DataFrame([
    {"slope_code": code, "slope_description": desc}
    for code, desc in slope_code_to_description.items()
])

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
topo_features_cleaning = topo_features_cleaning.applymap(remove_accents)

# Final cleaned topo features table (referencing slope_code, not description)
topo_features_clean = topo_features_cleaning[[
    'topo_id',
    'ID',
    #'profile', 
    'slope_code',
    'altitude',
    'aspect',
    'land_surface_temp',
    'dem_elevation'
]]

# Preview
topo_features_clean.head()


,topo_id,ID,slope_code,altitude,aspect,land_surface_temp,dem_elevation
0,1,2770,NaN,32.0,<NA>,<NA>,<NA>
1,2,48,D5,NaN,<NA>,<NA>,<NA>
2,3,1618,NaN,NaN,<NA>,<NA>,<NA>
3,4,881,D1,1210.0,<NA>,<NA>,<NA>
4,5,1750,NaN,NaN,<NA>,<NA>,<NA>


In [69]:
# save clean csv table
topo_features_clean.to_csv("/Users/inesschwartz/Desktop/topo_features_clean.csv", index=False)

# Geological features table

In [70]:
# Load cleaned geo features data
geo_features_cleaning = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Perfis_local.xlsx")

# Rename columns
geo_features_cleaning.rename(columns={
    'PERFIL': 'profile',
    'GEOLOGIA': 'geology_id',
    'LITOLOGIA': 'lithology_id',
    'LITOLOGIA_1954': 'lithology_1954_id',
}, inplace=True)

# Add primary key column
geo_features_cleaning.insert(0, 'geo_features_id', range(1, len(geo_features_cleaning) + 1))

# Final normalized geo_features table (with codes as foreign keys)
geo_features_clean = geo_features_cleaning[[
    'geo_features_id',
    #'profile',
    'ID',
    'geology_id',
    'lithology_id',
    'lithology_1954_id'
]]

# ensure consistent profile formatting
#geo_features_clean['profile'] = geo_features_clean['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
geo_features_clean = geo_features_clean.applymap(remove_accents)

#preview
geo_features_clean.head()

,geo_features_id,ID,geology_id,lithology_id,lithology_1954_id
0,1,2770,NaN,d,NaN
1,2,48,Oendolongo,pp,Sistema do Maiombe
2,3,1618,NaN,NaN,NaN
3,4,881,Karroo,Cs/Cal,Serie de Cassanje - T2'T1
4,5,1750,NaN,NaN,NaN


In [71]:
# save clean csv table
geo_features_clean.to_csv("/Users/inesschwartz/Desktop/geo_features_clean.csv", index=False)

In [73]:
## mapping tables
# Mappings for geology
geology_mapping = {
    "Kalahari": "Sistema do Kalahari",
    "Superficiais": "Formações Superficiais",
    "Karroo": "Sistema do Karroo",
    "Bembe": "Sistema do Bembe",
    "Oendolongo": "Sistema do Oendolongo",
    "Base": "Complexo de base",
    "Proterozóico": "Proterozóico",
    "Pleistocénico": "Pleistocénico",
    "Terciário": "Terciário (médio e inferior)",
    "TQ": "Quaternário e Terciário superior",
    "Cretácio": "",  # no description provided
    "RPKS": "Recente Plistocénico e Kalahari Superior"
}

# Mappings for lithology_1954
lithology_1954_mapping = {
    "γ": "Granitos, Granodioritos e Quartzodioritos",
    "PL": "Xistos, metaquartzitos, conglomerados, arcoses, ect.",
    "λ": "Rochas eruptivas indeterminadas",
    "δp": "Doleritos, doleritos pigeoníticos",
    "δab": "Diabases, diabases albito-cloriticas",
    "ε": "Noritos, gabros e peridotitos",
    "JK": "Composto de conglomerados, areias, cascalhos do Kalahari",
    "C": "Série Xisto - calcária",
    "Cal": "Sedimentos arenosos não consolidados",
    "K": "Série xisto - gresosa",
    "RT": "Não diferenciado",
    "σ": "Sienitos, sienitos nefelínicos",
    "Q": "Depósitos fossilíferos",
    "CS": "Grande conglomerado e série de Mwashya"
}

# Mappings for lithology
lithology_mapping = {
    "a": "Rochas arenáceas consolidadas",
    "aq": "Grés quartzíticos do Oendolongo",
    "b": "Rochas eruptivas básicas",
    "c": "Rochas sedimentares consolidadas calcárias",
    "c'": "Rochas sedimentares não consolidadas calcárias",
    "cg": "Rochas cristalofílicas argiláceas",
    "d": "Sedimentos não consolidados de origem marinha",
    "dc": "Depósitos coluvionares",
    "dr": "Diorítos",
    "e": "Rochas sedimentares consolidadas não calcárias",
    "g": "Rochas argiláceas consolidadas não calcárias",
    "g'": "Rochas argiláceas não consolidadas não calcárias",
    "g''": "Rochas cristalinas pouco micas em quartzo",
    "gp": "Rochas do  complexo gabro-plagioclastíco",
    "k": "Sedimentos não consolidados grosseiros do Kalahari",
    "m": "Rochas sedimentares não consolidadas calco-gipsíferas",
    "m'": "Depósitos coluvionares margosos",
    "mm": "Materiais mistos",
    "n": "Sedimentos não consolidados de origem continental",
    "nd": "não descrito",
    "pp": "Sedimentos não consolidados grosseiros plio-plistocénicos",
    "q": "Rochas cristalinas quartzíferas",
    "q'": "Materiais redistribuídos provenientes de desagregação rochas crist. quartzíferas",
    "qf": "Quartzitos ferruginosos do Oendolongo",
    "r": "Sedimentos grosseiros não especificados",
    "s": "Sienitos",
    "sx": "Formações (ou rochas) sedimentares não especificadas",
    "sx1": "Rochas sedimentares consolidadas com e sem calcário",
    "sx2": "Rochas sedimentares consolidadas",
    "v": "Materiais vulcânicos",
    "v'": "Rochas do complexo alcalino e/ou carboatítico",
    "x": "Rochas consolidadas não especificadas",
    "xm": "Xistos metamórficos",
    "xq": "Rochas cristalinas não especificadas",
    "z": "Rochas metassedimentares"
}

# Save each mapping as a DataFrame
pd.DataFrame([
    {"geology_code": k, "geology_description": v}
    for k, v in geology_mapping.items()
]).to_csv("/Users/inesschwartz/Desktop/geology_mapping.csv", index=False)

pd.DataFrame([
    {"lithology_code": k, "lithology_description": v}
    for k, v in lithology_mapping.items()
]).to_csv("/Users/inesschwartz/Desktop/lithology_mapping.csv", index=False)

pd.DataFrame([
    {"lithology_1954_code": k, "lithology_1954_description": v}
    for k, v in lithology_1954_mapping.items()
]).to_csv("/Users/inesschwartz/Desktop/lithology1954_mapping.csv", index=False)


# Minerology info table

# Biology Info table